[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arvidl/ELMED219-2026/blob/main/Lab3-GenAI-LLM/notebooks/09-chatgpt-claude-api.ipynb)

# ChatGPT og Claude API - Praktisk Integrasjon

**ELMED219 / BMED365 - Lab 3**

---

> **Merk**: Denne notebooken er valgfri og krever API-nøkler fra OpenAI og/eller Anthropic. Den er beregnet på studenter som ønsker å eksperimentere med programmatisk bruk av LLM-er.

## Læringsmål

Etter denne notebooken skal du kunne:
- Sette opp og bruke OpenAI (ChatGPT) og Anthropic (Claude) APIer
- Implementere feilhåndtering og rate limiting
- Bygge en enkel medisinsk assistent-funksjon
- Forstå beste praksis for API-bruk i medisinske kontekster

## Innhold

1. [Oppsett og konfigurasjon](#1-oppsett-og-konfigurasjon)
2. [OpenAI API (ChatGPT)](#2-openai-api-chatgpt)
3. [Anthropic API (Claude)](#3-anthropic-api-claude)
4. [Feilhåndtering](#4-feilhåndtering)
5. [Praktiske eksempler](#5-praktiske-eksempler)
6. [Sikkerhetshensyn](#6-sikkerhetshensyn)

---

## 1. Oppsett og konfigurasjon

### Forutsetninger

1. API-nøkkel fra [OpenAI](https://platform.openai.com/api-keys) og/eller [Anthropic](https://console.anthropic.com/)
2. Python-pakker: `openai`, `anthropic`

### Beste praksis for API-nøkler

**Aldri** hardkod API-nøkler i kode. Bruk miljøvariabler:

```bash
# I terminal:
export OPENAI_API_KEY="sk-..."
export ANTHROPIC_API_KEY="sk-ant-..."
```

Eller en `.env`-fil (som IKKE skal committes til git).

In [2]:
# Installer nødvendige pakker (kjør om nødvendig)
# !pip install openai anthropic python-dotenv

import os
import sys

# Sjekk om vi er i Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Kjører i Google Colab")
    !pip install openai anthropic python-dotenv --quiet

# Last miljøvariabler fra .env hvis den finnes
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Miljøvariabler lastet fra .env")
except ImportError:
    print("python-dotenv ikke installert - bruker eksisterende miljøvariabler")

# Sjekk tilgjengelige APIer
OPENAI_AVAILABLE = bool(os.getenv("OPENAI_API_KEY"))
ANTHROPIC_AVAILABLE = bool(os.getenv("ANTHROPIC_API_KEY"))

print(f"\nAPI-tilgjengelighet:")
print(f"  OpenAI (ChatGPT): {'Tilgjengelig' if OPENAI_AVAILABLE else 'Ikke konfigurert'}")
print(f"  Anthropic (Claude): {'Tilgjengelig' if ANTHROPIC_AVAILABLE else 'Ikke konfigurert'}")

if not (OPENAI_AVAILABLE or ANTHROPIC_AVAILABLE):
    print("\n⚠️  Ingen API-nøkler funnet.")
    print("   Eksemplene vil vise simulerte responser.")

Miljøvariabler lastet fra .env

API-tilgjengelighet:
  OpenAI (ChatGPT): Tilgjengelig
  Anthropic (Claude): Ikke konfigurert


---

## 2. OpenAI API (ChatGPT)

In [3]:
from typing import Optional

def chat_with_gpt(prompt: str, 
                  model: str = "gpt-3.5-turbo",
                  temperature: float = 0.7,
                  system_prompt: str = None) -> str:
    """
    Send en melding til ChatGPT og få respons.
    
    Args:
        prompt: Brukerens melding
        model: GPT-modell (gpt-3.5-turbo, gpt-4, etc.)
        temperature: 0-2, hvor 0 er deterministisk
        system_prompt: Valgfri system-instruksjon
    
    Returns:
        Modellens respons
    """
    if not OPENAI_AVAILABLE:
        return f"[SIMULERT RESPONS]\nDette er en simulert respons for: {prompt[:100]}..."
    
    try:
        from openai import OpenAI
        client = OpenAI()
        
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Feil ved API-kall: {e}"

# Test
respons = chat_with_gpt(
    "Forklar kort hva HbA1c måler.",
    temperature=0.3,
    system_prompt="Du er en hjelpsom medisinsk assistent. Svar kort og presist."
)
print("Respons:")
print(respons)

Respons:
HbA1c måler gjennomsnittlig blodsukkernivå over de siste 2-3 månedene.


---

## 3. Anthropic API (Claude)

In [4]:
def chat_with_claude(prompt: str,
                     model: str = "claude-3-sonnet-20240229",
                     temperature: float = 0.7,
                     system_prompt: str = None) -> str:
    """
    Send en melding til Claude og få respons.
    
    Args:
        prompt: Brukerens melding
        model: Claude-modell (claude-3-sonnet, claude-3-opus, etc.)
        temperature: 0-1
        system_prompt: Valgfri system-instruksjon
    
    Returns:
        Modellens respons
    """
    if not ANTHROPIC_AVAILABLE:
        return f"[SIMULERT RESPONS]\nDette er en simulert respons for: {prompt[:100]}..."
    
    try:
        from anthropic import Anthropic
        client = Anthropic()
        
        kwargs = {
            "model": model,
            "max_tokens": 1024,
            "temperature": temperature,
            "messages": [{"role": "user", "content": prompt}]
        }
        
        if system_prompt:
            kwargs["system"] = system_prompt
        
        response = client.messages.create(**kwargs)
        
        return response.content[0].text
        
    except Exception as e:
        return f"Feil ved API-kall: {e}"

# Test
respons = chat_with_claude(
    "Hva er forskjellen mellom Type 1 og Type 2 diabetes?",
    temperature=0.3,
    system_prompt="Du er en medisinsk assistent. Forklar på en pasientvennlig måte."
)
print("Respons:")
print(respons)

Respons:
[SIMULERT RESPONS]
Dette er en simulert respons for: Hva er forskjellen mellom Type 1 og Type 2 diabetes?...


---

## 4. Feilhåndtering

Ved programmatisk bruk av APIer er robust feilhåndtering kritisk.

In [5]:
import time

class RobustAPIClient:
    """
    Wrapper med retry-logikk og rate limiting.
    """
    
    def __init__(self, max_retries: int = 3, base_delay: float = 1.0):
        self.max_retries = max_retries
        self.base_delay = base_delay
        self.last_call_time = 0
        self.min_interval = 0.5  # Minimum tid mellom kall (sekunder)
    
    def _wait_if_needed(self):
        """Respekter rate limits."""
        elapsed = time.time() - self.last_call_time
        if elapsed < self.min_interval:
            time.sleep(self.min_interval - elapsed)
        self.last_call_time = time.time()
    
    def call_api(self, func, *args, **kwargs):
        """
        Utfør API-kall med retry-logikk.
        """
        for attempt in range(self.max_retries):
            try:
                self._wait_if_needed()
                return func(*args, **kwargs)
                
            except Exception as e:
                error_msg = str(e)
                
                # Sjekk om det er en rate limit-feil
                if "rate_limit" in error_msg.lower() or "429" in error_msg:
                    wait_time = self.base_delay * (2 ** attempt)
                    print(f"Rate limit - venter {wait_time}s før retry {attempt + 1}/{self.max_retries}")
                    time.sleep(wait_time)
                else:
                    # Andre feil - returner feilmelding
                    return f"Feil etter {attempt + 1} forsøk: {e}"
        
        return "Maksimalt antall forsøk nådd"

# Eksempel på bruk
print("RobustAPIClient klar for bruk.")
print("Denne klassen håndterer rate limits og nettverksfeil automatisk.")

RobustAPIClient klar for bruk.
Denne klassen håndterer rate limits og nettverksfeil automatisk.


---

## 5. Praktiske eksempler

### Eksempel: Medisinsk assistent-funksjon

In [6]:
def medisinsk_assistent(sporsmal: str, kontekst: str = None) -> dict:
    """
    Medisinsk assistent med sikkerhetsinstruksjoner.
    
    Returns:
        Dict med 'svar' og 'advarsler'
    """
    
    system_prompt = """Du er en medisinsk informasjonsassistent.

VIKTIGE REGLER:
1. Gi kun generell helseinformasjon
2. Aldri gi spesifikke diagnoser eller behandlingsanbefalinger
3. Anbefal alltid å konsultere lege for personlige helsebekymringer
4. Vær tydelig på begrensninger i din kunnskap
5. Ved akutte symptomer: Anbefal umiddelbart å kontakte helsepersonell

Svar på norsk, kort og presist."""
    
    full_prompt = sporsmal
    if kontekst:
        full_prompt = f"Kontekst: {kontekst}\n\nSpørsmål: {sporsmal}"
    
    # Her ville vi normalt kalt API
    # For demonstrasjon, simulerer vi responsen
    
    simulert_svar = {
        "svar": f"[Simulert svar for: {sporsmal[:50]}...]\n\n" + 
                "Dette er generell helseinformasjon. " +
                "For personlig vurdering, kontakt helsepersonell.",
        "advarsler": [
            "Denne informasjonen erstatter ikke medisinsk rådgivning",
            "Konsulter lege for personlig vurdering"
        ]
    }
    
    return simulert_svar

# Test
resultat = medisinsk_assistent(
    "Hva er vanlige bivirkninger av metformin?",
    kontekst="Pasient med nyoppdaget diabetes type 2"
)

print("MEDISINSK ASSISTENT")
print("=" * 50)
print(f"\nSvar:\n{resultat['svar']}")
print(f"\nAdvarsler:")
for adv in resultat['advarsler']:
    print(f"  ⚠️  {adv}")

MEDISINSK ASSISTENT

Svar:
[Simulert svar for: Hva er vanlige bivirkninger av metformin?...]

Dette er generell helseinformasjon. For personlig vurdering, kontakt helsepersonell.

Advarsler:
  ⚠️  Denne informasjonen erstatter ikke medisinsk rådgivning
  ⚠️  Konsulter lege for personlig vurdering


---

## 6. Sikkerhetshensyn

### Ved bruk av LLM-APIer i helsekontekst:

**1. Personvern**
- Aldri send reelle pasientdata til eksterne APIer uten godkjenning
- Vurder lokale modeller (Ollama, etc.) for sensitiv data
- Sjekk databehandleravtaler med API-leverandør

**2. Hallusinering**
- LLM-er kan finne på fakta
- Verifiser alltid medisinsk informasjon
- Bruk lav temperature for faktabaserte spørsmål

**3. Logging**
- Logg alle API-kall for sporbarhet
- Slett logger med pasientinformasjon i henhold til policy

**4. Menneskelig tilsyn**
- AI-output skal alltid vurderes av helsepersonell
- Implementer tydelige advarsler i brukergrensesnitt

---

## Oppsummering

### Hovedpunkter

1. **API-nøkler** skal aldri hardkodes - bruk miljøvariabler
2. **Feilhåndtering** med retry-logikk er essensielt
3. **System prompts** med sikkerhetsinstruksjoner er kritisk for medisinsk bruk
4. **Personvern** må ivaretas - vurder lokale modeller for sensitiv data
5. **Menneskelig tilsyn** er alltid påkrevd for medisinsk AI

### Ressurser

- [OpenAI API dokumentasjon](https://platform.openai.com/docs)
- [Anthropic API dokumentasjon](https://docs.anthropic.com)
- [OpenAI Cookbook](https://cookbook.openai.com/)

---

*Tilbake til [Lab 3 oversikt](../README.md)*